In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=25b49211df0da15fec5d3ccc3a77917801b1a039fdb426bcf8c23f8a3b92bd05
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [ ]:
import openai
import pandas as pd

# --------------------------------------------------
# Kylers amazing ML Idea: 
# Use a generative pre trained model. 
# I'm curious if we fed it batch csv data per vendor and then made it generate summaries 
# Limit the summaries to under 250 words. 
# Start checking the data and see if it works
# if it does, throw it on the checkbook. 
# "powered by AI" take that socrata. 
# Developed by Vartan A. 
# See ReadMe On the repo for code explantion 
# --------------------------------------------------


#First things first, we have to set up the OpenAI API key and model ID. Decided to use gpt3 because its powerful and awesome. 
openai.api_key = "OPENAI_API_KEY"
model_engine = "text-davinci-002"

# Second thing, we need to define a function to generate summaries using GPT-3 Pre-trained model. 
def generate_summaries(batches):
    prompt = "Please summarize the following text into 250 words or less:\n"
    prompt += "\n".join(["\n".join(batch) + "\n" for batch in batches]) + "\nSummary:"
    responses = openai.Completion.create(
        engine=model_engine,
        prompt=prompt,
        max_tokens=512,
        n=len(batches),
        stop=None,
        temperature=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=0
    )
    summaries = []
    for i, choice in enumerate(responses.choices):
        start = sum([len(batch) for batch in batches[:i]])
        end = start + len(batches[i])
        summaries += [choice.text.strip()] * len(batches[i])
    return summaries

# Third, read the CSV data and generate summaries using GPT-3. 
for vendor in ['/content/vendor1.csv']:
    data = pd.read_csv(vendor)
    data = data.astype(str)

    # Group data by account, fund, vendor, description, item, and department columns
    # TODO: Inside the struct list just add Vendor for the vendors name from the column list if a csv has it.
    grouped_data = data.groupby(['Account', 'Fund', 'Desc', 'Item', 'Dept'])

    # Next, summarize data for each group
    summaries = []
    batch_size = 5
    for group, group_data in grouped_data:
        batch = group_data['Item'].tolist()
        summaries += generate_summaries([batch])

    # At last, write the summaries to a new CSV file
    data['summary'] = summaries
    data.to_csv(vendor[:-4]+'summarize.csv', index=False)

